# Import the necessary libraries

In [8]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.6.1-cp313-cp313-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp313-cp313-win_amd64.whl (11.1 MB)
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/41.0 MB ? eta -:--:--
   ----

In [12]:
pip install wordcloud

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

 # Read the DataFrames from CSV files

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


# DataFrames Information

In [ ]:
reddit_data

In [ ]:
twitter_data

In [ ]:
reddit_data.info()

In [ ]:
twitter_data.info()

In [ ]:
#renaming the columns
twitter_data.columns=['messages','labels']
reddit_data.columns=['messages','labels']

In [ ]:
twitter_data.head()

In [ ]:
reddit_data.head()

# Merge DataFrames

In [ ]:
data = pd.concat([twitter_data, reddit_data], ignore_index = True)

In [ ]:
data

In [ ]:
data.tail()

In [ ]:
data.info()

 # Handling Missing Values

In [ ]:
data.isnull().sum()

In [ ]:
data = data.dropna()

In [ ]:
data.head()

In [ ]:
data.tail()

# Data Preprocessing

In [ ]:
#Libraries of NLTK
import re,string,nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer,WordNetLemmatizer

In [ ]:
#cleaning the data from unnecessary data

#replacing url with string URL
def replace_url(text):
    return re.sub('https?:\/\/\S*|www\.\S+','URL',text)

#removing html
def remove_html(text):
    return re.sub('<.*?>','',text)

#replacing mentions with string user
def replace_mentions(text):
    return re.sub('@\S*','user',text,flags=re.IGNORECASE)

#replacing numbers with string number
def replace_num(text):
    return re.sub('^[+-]*?\d{1,3}[- ]*?\d{1,10}|\d{10}','NUMBER',text)

#replacing <3 with sring heart
def replace_heart(text):
    return re.sub('<3','HEART', text)

#removing alphanumeric characters eg-XYZ123ABC
def remove_alphanumeric(text):
    return re.sub('\w*\d+\w*','',text)

#removing all english stopwords 
def remove_stopwords(text):
    text = ' '.join([word for word in text.split() if word not in stopwords.words("english")])
    return text

#removing punctuations
def remove_punctuations(text):
    text=''.join([word for word in text if word not in string.punctuation])
    return text

#reducing words to thier root form
def lemmatization(text):
    lm= WordNetLemmatizer()
    text = ' '.join([lm.lemmatize(word, pos='v') for word in text.split()])
    return text

In [ ]:
def clean_text(text):
    text=str(text).lower()
    text = replace_url(text)
    text = remove_html(text)
    text = replace_mentions(text)
    text = replace_num(text)
    text = replace_heart(text)
    text = remove_alphanumeric(text)
    text = remove_stopwords(text)
    text=remove_punctuations(text)
    #text=stemming(text)
    text=lemmatization(text)
    return text

In [ ]:
data['messages1']=data['messages'].apply(clean_text)

In [ ]:
data['messages1']

# Sentiment Analysis

In [ ]:
import nltk
nltk.download('vader_lexicon')

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize the VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Function to get sentiment scores
def get_sentiment_scores(text):
    # Get sentiment scores using VADER
    scores = sid.polarity_scores(text)
    # Extract compound score, which represents overall sentiment
    compound_score = scores['compound']
    # Return compound score
    return compound_score

# Apply sentiment analysis to the cleaned text data
data['sentiment_score'] = data['messages1'].apply(get_sentiment_scores)


In [ ]:
data.info()

In [ ]:
data.drop('messages', axis=1, inplace=True)

In [ ]:
data.head()

# EDA & Data Visualisation

In [ ]:
# The distribution of sentiments
data.groupby('labels').count().plot(kind='bar')

In [ ]:
data['label'] = data['labels'].map({-1.0:'Negative', 0.0:'Neutral', 1.0:'Positive'})

In [ ]:
#Plotting the distribution of text length for positive sentiment Data
fig = plt.figure(figsize=(14,7))
data['length'] = data.messages1.str.split().apply(len)
ax1 = fig.add_subplot(122)
sns.histplot(data[data['label']=='Positive']['length'], ax=ax1,color='green')
describe = data.length[data.label=='Positive'].describe().to_frame().round(2)

ax2 = fig.add_subplot(121)
ax2.axis('off')
font_size = 14
bbox = [0, 0, 1, 1]
table = ax2.table(cellText = describe.values, rowLabels = describe.index, bbox=bbox, colLabels=describe.columns)
table.set_fontsize(font_size)
fig.suptitle('Distribution of text length for positive sentiment tweets.', fontsize=16)

plt.show()

In [ ]:
#Plotting the distribution of text length for Negative sentiment Data
fig = plt.figure(figsize=(14,7))
data['length'] = data.messages1.str.split().apply(len)
ax1 = fig.add_subplot(122)
sns.histplot(data[data['label']=='Negative']['length'], ax=ax1,color='red')
describe = data.length[data.label=='Negative'].describe().to_frame().round(2)

ax2 = fig.add_subplot(121)
ax2.axis('off')
font_size = 14
bbox = [0, 0, 1, 1]
table = ax2.table(cellText = describe.values, rowLabels = describe.index, bbox=bbox, colLabels=describe.columns)
table.set_fontsize(font_size)
fig.suptitle('Distribution of text length for negative sentiment tweets.', fontsize=16)

plt.show()

In [ ]:
#Plotting the distribution of text length for Neutral sentiment Data
fig = plt.figure(figsize=(14,7))
data['length'] = data.messages1.str.split().apply(len)
ax1 = fig.add_subplot(122)
sns.histplot(data[data['label']=='Neutral']['length'], ax=ax1,color='red')
describe = data.length[data.label=='Neutral'].describe().to_frame().round(2)

ax2 = fig.add_subplot(121)
ax2.axis('off')
font_size = 14
bbox = [0, 0, 1, 1]
table = ax2.table(cellText = describe.values, rowLabels = describe.index, bbox=bbox, colLabels=describe.columns)
table.set_fontsize(font_size)
fig.suptitle('Distribution of text length for neutral sentiment tweets.', fontsize=16)

plt.show()

In [ ]:
import plotly.express as px
fig = px.pie(data, names='label', title ='Pie chart of different sentiments of data')
fig.show()

In [ ]:
# data.drop(['length'], axis=1, inplace=True)
# data.head

In [ ]:
data.info()

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
def wordcount_gen(df, category, sentiment_column='sentiment_score'):
              """
              Generating Word Cloud

              Inputs:
                - df: DataFrame containing the data
                - category: Positive/Negative/Neutral
                - sentiment_column (optional): Name of the column containing sentiment score (default: 'sentiment_score')
              """

              # Filter data based on sentiment
              filtered_data = df[df[sentiment_column] > 0.5] if category == 'Positive' else df[df[sentiment_column] <= 0.5]

              # Combine all messages from the filtered data
              combined_tweets = " ".join(filtered_data['messages1'])

              # Initialize wordcloud object
              wc = WordCloud(background_color='black',
                              max_words=50,
                              stopwords=STOPWORDS)

              # Generate and plot wordcloud
              plt.figure(figsize=(10, 10))
              plt.imshow(wc.generate(combined_tweets))
              plt.title('{} Sentiment Words'.format(category), fontsize=20)
              plt.axis('off')
              plt.show()
#Positive tweet words
wordcount_gen(data, 'Positive')

# Negative tweet words
wordcount_gen(data, 'Negative')

# Neutral tweet words (assuming sentiment_column < 0.5 is neutral)
wordcount_gen(data, 'Neutral')

       

In [ ]:
# data.drop(['label'], axis=1, inplace=True)
# data.head()

In [ ]:
# Visualize sentiment distribution using a histogram
plt.figure(figsize=(8, 6))
sns.histplot(data['sentiment_score'], bins=20, kde=True, color='skyblue')
plt.title('Sentiment Distribution')
plt.xlabel('Sentiment Score')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


In [ ]:
# Compare sentiment across labels 
plt.figure(figsize=(10, 6))
sns.boxplot(x='labels', y='sentiment_score', data=data, palette='Set2')
plt.title('Sentiment Across Topics')
plt.xlabel('Topic')
plt.ylabel('Sentiment Score')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Detect anomalies or outliers in sentiment scores
plt.figure(figsize=(8, 6))
sns.boxplot(data['sentiment_score'], color='lightgreen')
plt.title('Detecting Anomalies in Sentiment Scores')
plt.xlabel('Sentiment Score')
plt.grid(True)
plt.show()

In [ ]:
#Graph of Stopwords
fig , ax = plt.subplots(figsize = (10,10))
ax = data['labels'].value_counts().plot(kind = 'bar')

plt.xticks(rotation = 0, size = 14)
plt.yticks(size = 14, color = 'white')
plt.title('Distribution of Sentiment', size = 20)

ax.annotate(text = data['labels'].value_counts().values[0], xy = (-0.13,88079), size = 18)
ax.annotate(text = data['labels'].value_counts().values[1], xy = (0.87,68253), size = 18)
ax.annotate(text = data['labels'].value_counts().values[2], xy = (1.87,43786), size = 18)

plt.show()

# Train & Test Split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [ ]:
data.info()

In [ ]:
#Finding X and y
X = data['messages1']
y = data['labels']


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [ ]:
##Vectorize text data
# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the testing data
X_test_tfidf = tfidf_vectorizer.transform(X_test)


# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Initialize logistic regression model
log_reg_model = LogisticRegression()

# Train the logistic regression model on the training data
log_reg_model.fit(X_train_tfidf, y_train)


In [ ]:
# Predict on the testing data
y_pred_log_reg = log_reg_model.predict(X_test_tfidf)

In [ ]:
# Calculate accuracy
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
print("Logistic Regression Accuracy:", accuracy_log_reg)

In [ ]:
# Generate confusion matrix
conf_matrix_log_reg = confusion_matrix(y_test, y_pred_log_reg)
print("Logistic Regression Confusion Matrix:")
print(conf_matrix_log_reg)


In [ ]:
#pip install scikit-plot

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt
# Plot normalized confusion matrix
skplt.metrics.plot_confusion_matrix(y_test, y_pred_log_reg, normalize=True)
plt.title("Normalized Confusion Matrix")
plt.show()

In [ ]:
# Generate classification report
report_log_reg = classification_report(y_test, y_pred_log_reg)
print("Logistic Regression Classification Report:")
print(report_log_reg)

# Support Vector Machine (SVM)

In [ ]:
from sklearn.svm import SVC

# Initialize SVM model
svm_model = SVC()

# Train the SVM model on the training data
svm_model.fit(X_train_tfidf, y_train)

In [ ]:
# Predict on the testing data
y_pred_svm = svm_model.predict(X_test_tfidf)


In [ ]:
# Calculate accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy:", accuracy_svm)

In [ ]:
# Generate confusion matrix
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
print("SVM Confusion Matrix:")
print(conf_matrix_svm)

In [ ]:
# Generate classification report
report_svm = classification_report(y_test, y_pred_svm)
print("SVM Classification Report:")
print(report_svm)


# DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Initialize Decision Tree Classifier model
dt_model = DecisionTreeClassifier(max_depth=3, min_samples_split=2)


# Train the Decision Tree Classifier model on the training data
dt_model.fit(X_train_tfidf, y_train)

# Predict on the testing data
y_pred_dt = dt_model.predict(X_test_tfidf)


In [ ]:
# Calculate accuracy
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Decision Tree Classifier Accuracy:", accuracy_dt)

# Generate confusion matrix
conf_matrix_dt = confusion_matrix(y_test, y_pred_dt)
print("Decision Tree Classifier Confusion Matrix:")
print(conf_matrix_dt)

In [ ]:
# Generate classification report
report_dt = classification_report(y_test, y_pred_dt)
print("Decision Tree Classifier Classification Report:")
print(report_dt)

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt
# Plot normalized confusion matrix
skplt.metrics.plot_confusion_matrix(y_test, y_pred_dt, normalize=True)
plt.title("Normalized Confusion Matrix")
plt.show()

# RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize RandomForest Classifier model
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10)

# Train the RandomForest Classifier model on the training data
rf_model.fit(X_train_tfidf, y_train)

In [ ]:
# Predict on the testing data
y_pred_rf = rf_model.predict(X_test_tfidf)


In [ ]:
# Calculate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("RandomForest Classifier Accuracy:", accuracy_rf)

# Generate confusion matrix
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
print("RandomForest Classifier Confusion Matrix:")
print(conf_matrix_rf)

# Generate classification report
report_rf = classification_report(y_test, y_pred_rf)
print("RandomForest Classifier Classification Report:")
print(report_rf)

# Support Vector Classifier

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Initialize SVC model
svc_model = SVC(kernel='linear', C=1.0)

# Train the SVC model on the training data
svc_model.fit(X_train_tfidf, y_train)

In [ ]:

# Predict on the testing data
y_pred_svc = svc_model.predict(X_test_tfidf)

# Calculate accuracy
accuracy_svc = accuracy_score(y_test, y_pred_svc)
print("SVC Accuracy:", accuracy_svc)


In [ ]:

# Generate confusion matrix
conf_matrix_svc = confusion_matrix(y_test, y_pred_svc)
print("SVC Confusion Matrix:")
print(conf_matrix_svc)

# Generate classification report
report_svc = classification_report(y_test, y_pred_svc)
print("SVC Classification Report:")
print(report_svc)

# KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Initialize KNeighborsClassifier model
knn_model = KNeighborsClassifier()

# Train the KNeighborsClassifier model on the training data
knn_model.fit(X_train_tfidf, y_train)


In [ ]:
# Predict on the testing data
y_pred_knn = knn_model.predict(X_test_tfidf)

# Calculate accuracy
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("KNeighborsClassifier Accuracy:", accuracy_knn)


In [ ]:

# Generate confusion matrix
conf_matrix_knn = confusion_matrix(y_test, y_pred_knn)
print("KNeighborsClassifier Confusion Matrix:")
print(conf_matrix_knn)

# Generate classification report
report_knn = classification_report(y_test, y_pred_knn)
print("KNeighborsClassifier Classification Report:")
print(report_knn)


In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt
# Plot normalized confusion matrix
skplt.metrics.plot_confusion_matrix(y_test, y_pred_knn, normalize=True)
plt.title("Normalized Confusion Matrix")
plt.show()

# GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Initialize GradientBoostingClassifier model
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1)

# Train the GradientBoostingClassifier model on the training data
gb_model.fit(X_train_tfidf, y_train)


In [ ]:
# Predict on the testing data
y_pred_gb = gb_model.predict(X_test_tfidf)

# Calculate accuracy
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print("GradientBoostingClassifier Accuracy:", accuracy_gb)


In [ ]:
# Generate confusion matrix
conf_matrix_gb = confusion_matrix(y_test, y_pred_gb)
print("GradientBoostingClassifier Confusion Matrix:")
print(conf_matrix_gb)

# Generate classification report
report_gb = classification_report(y_test, y_pred_gb)
print("GradientBoostingClassifier Classification Report:")
print(report_gb)

# AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Initialize AdaBoostClassifier model
ada_model = AdaBoostClassifier(n_estimators=50, learning_rate=1.0)

# Train the AdaBoostClassifier model on the training data
ada_model.fit(X_train_tfidf, y_train)

In [ ]:
# Predict on the testing data
y_pred_ada = ada_model.predict(X_test_tfidf)

# Calculate accuracy
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print("AdaBoostClassifier Accuracy:", accuracy_ada)

In [ ]:
# Generate confusion matrix
conf_matrix_ada = confusion_matrix(y_test, y_pred_ada)
print("AdaBoostClassifier Confusion Matrix:")
print(conf_matrix_ada)

# Generate classification report
report_ada = classification_report(y_test, y_pred_ada)
print("AdaBoostClassifier Classification Report:")
print(report_ada)


# XGBClassifier

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Initialize XGBClassifier model
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1)

# Train the XGBClassifier model on the training data
xgb_model.fit(X_train_tfidf, y_train)


In [ ]:
# Predict on the testing data
y_pred_xgb = xgb_model.predict(X_test_tfidf)

# Calculate accuracy
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("XGBClassifier Accuracy:", accuracy_xgb)


In [ ]:
# Generate confusion matrix
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
print("XGBClassifier Confusion Matrix:")
print(conf_matrix_xgb)

# Generate classification report
report_xgb = classification_report(y_test, y_pred_xgb)
print("XGBClassifier Classification Report:")
print(report_xgb)


# LGBMClassifier

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Initialize LGBMClassifier model
lgbm_model = LGBMClassifier(n_estimators=100, learning_rate=0.1)

# Train the LGBMClassifier model on the training data
lgbm_model.fit(X_train_tfidf, y_train)


In [ ]:
# Predict on the testing data
y_pred_lgbm = lgbm_model.predict(X_test_tfidf)

# Calculate accuracy
accuracy_lgbm = accuracy_score(y_test, y_pred_lgbm)
print("LGBMClassifier Accuracy:", accuracy_lgbm)


In [ ]:
# Generate confusion matrix
conf_matrix_lgbm = confusion_matrix(y_test, y_pred_lgbm)
print("LGBMClassifier Confusion Matrix:")
print(conf_matrix_lgbm)

# Generate classification report
report_lgbm = classification_report(y_test, y_pred_lgbm)
print("LGBMClassifier Classification Report:")
print(report_lgbm)

#  GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Initialize GaussianNB model
nb_model = GaussianNB()

# Train the GaussianNB model on the training data
nb_model.fit(X_train_tfidf.toarray(), y_train)  # GaussianNB expects dense matrix


In [ ]:
# Predict on the testing data
y_pred_nb = nb_model.predict(X_test_tfidf.toarray())

# Calculate accuracy
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print("GaussianNB Accuracy:", accuracy_nb)


In [ ]:
# Generate confusion matrix
conf_matrix_nb = confusion_matrix(y_test, y_pred_nb)
print("GaussianNB Confusion Matrix:")
print(conf_matrix_nb)

# Generate classification report
report_nb = classification_report(y_test, y_pred_nb)
print("GaussianNB Classification Report:")
print(report_nb)


# Multi-Layer Perceptron Classifier

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Initialize MLPClassifier model
mlp_model = MLPClassifier()

# Train the MLPClassifier model on the training data
mlp_model.fit(X_train_tfidf, y_train)


In [ ]:
# Predict on the testing data
y_pred_mlp = mlp_model.predict(X_test_tfidf)

# Calculate accuracy
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
print("MLPClassifier Accuracy:", accuracy_mlp)

In [ ]:
# Generate confusion matrix
conf_matrix_mlp = confusion_matrix(y_test, y_pred_mlp)
print("MLPClassifier Confusion Matrix:")
print(conf_matrix_mlp)

# Generate classification report
report_mlp = classification_report(y_test, y_pred_mlp)
print("MLPClassifier Classification Report:")
print(report_mlp)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

classifiers = {
    "LogisticRegression": LogisticRegression(),
    "SVC": SVC(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "GradientBoostingClassifier": GradientBoostingClassifier(),
    "AdaBoostClassifier": AdaBoostClassifier(),
    "XGBClassifier": XGBClassifier(),
    "LGBMClassifier": LGBMClassifier(),
    "GaussianNB": GaussianNB(),
    "MLPClassifier": MLPClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier()
}

# Loop through each classifier
for name, classifier in classifiers.items():
    # Train the classifier
    classifier.fit(X_train, y_train)
    
    # Make predictions
    y_pred = classifier.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy}")
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"{name} Confusion Matrix:")
    print(conf_matrix)
    
    # Generate classification report
    report = classification_report(y_test, y_pred)
    print(f"{name} Classification Report:")
    print(report)
    print()
